Import the libraries

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
from fastapi import FastAPI, Request, Response

Indicate pandas to display float numbers using only 2 decimals

In [5]:
pd.set_option('display.float_format', '{:.2f}'.format)

# 1. ETL

### 1.1 Extraction: start by retrieving the data from various csv files.

##### 1.1.1 Create all the necessary dataframes

In [ ]:
df1_ratings = pd.read_csv("https://drive.google.com/u/0/uc?id=1BAMVkLqMrXJDYNaLC973Jcd_r77WCgQn&export=download")
df2_ratings = pd.read_csv("https://drive.google.com/u/0/uc?id=1Pc60nR9sxXfmPIsJRvfhqYYOO5zA3VsT&export=download")
df3_ratings = pd.read_csv("https://drive.google.com/u/0/uc?id=1tkMrm_--FZWps1uXMwEF3XT_T3PMVQlk&export=download")
df4_ratings = pd.read_csv("https://drive.google.com/u/0/uc?id=167GBJwlhh58mVK-mSyJacQA8SB5uup93&export=download")
df5_ratings = pd.read_csv("https://drive.google.com/u/0/uc?id=1w94ESSIUruinU_Pf--jkKk8B2-GDxRb6&export=download")
df6_ratings = pd.read_csv("https://drive.google.com/u/0/uc?id=1N9MqB9DNpdGMWiK_fGXRf3FrosCjR9ob&export=download")
df7_ratings = pd.read_csv("https://drive.google.com/u/0/uc?id=16ipx0P4WY3SoVFN2LbVst1N_5yr98OYA&export=download")
df8_ratings = pd.read_csv("https://drive.google.com/u/0/uc?id=1n8UpCl-qWXPvzrhmP7W9ISH9lyy-_K4V&export=download")
df_amazon   = pd.read_csv("https://drive.google.com/u/0/uc?id=1WAtbMW-d49_PPSQ6Kngq9NoD-dcOOiC9&export=download")
df_disney   = pd.read_csv("https://drive.google.com/u/0/uc?id=1Qn_Tv5vEmWg5PRqJkBqN_0SaSG0zFtuR&export=download")
df_hulu     = pd.read_csv("https://drive.google.com/u/0/uc?id=11FFXj7Dn22HjAnT3hk0XmTzav7jY9B0l&export=download")
df_netflix  = pd.read_csv("https://drive.google.com/u/0/uc?id=1cwzLeYjPIeJ5aJbyJO-F1QO5iHYIh6T2&export=download")

##### 1.1.2 Combine all the ratings datasets

In [ ]:
df_ratings = pd.concat([df1_ratings, df2_ratings, df3_ratings, df4_ratings, 
                        df5_ratings, df6_ratings, df7_ratings, df8_ratings])

### 1.2 Transformation: basic EDA and data cleaning/preparation.

### _First group of datasets  (ratings 1 - 8)_

##### 1.2.1 Checking for null values

In [ ]:
df_ratings.isnull().sum()

userId       0
rating       0
timestamp    0
movieId      0
dtype: int64

##### 1.2.2 Checking and dropping duplicates

In [ ]:
df_ratings.duplicated().sum()
df_ratings.drop_duplicates(inplace=True)

##### 1.2.3 Checking data types of each column.

In [ ]:
df_ratings.dtypes

userId         int64
rating       float64
timestamp      int64
movieId       object
dtype: object

##### 1.2.4 Create a new column called date with a proper date format. I will NOT drop the timestamp column for now.

In [ ]:
df_ratings['date'] = pd.to_datetime(df_ratings['timestamp'], unit='s').dt.strftime('%Y-%m-%d')

##### 1.2.5 Get the average rating grouped by unique movies and store the array in a variable

In [ ]:
average_score = df_ratings.groupby('movieId')['rating'].mean()

In [ ]:
average_score.describe()

count   22998.00
mean        3.53
std         0.05
min         3.34
25%         3.50
50%         3.53
75%         3.57
max         3.72
Name: rating, dtype: float64

##### 1.2.6 Make a new dataframe containing the average rating and the movie ID from the previous variable by resetting the index. This dataframe will be used later on to create a bigger dataframe with more data.

In [ ]:
df_average_score = average_score.reset_index()[['movieId', 'rating']]

### _Second group of datasets (information about movies on Amazon, Disney, Hulu and Netflix)._

##### 1.2.7 Check for duplicates

In [ ]:
print(df_amazon.duplicated().sum())
print(df_disney.duplicated().sum())
print(df_hulu.duplicated().sum())
print(df_netflix.duplicated().sum())

0
0
0
0


##### 1.2.8 Make a list of the platforms dataframes

In [ ]:
platforms = [df_amazon, df_disney, df_hulu, df_netflix]

##### 1.2.9 Assing a name to these dataframes to create a composite ID later

In [ ]:
df_amazon.name = 'amazon'
df_disney.name = 'disney'
df_hulu.name = 'hulu'
df_netflix.name = 'netflix'

##### 1.2.10 Create a new column at the start of each of the platforms dataframes with the name 'id' and a value corresponding of the fist letter of the name of the platform and the show_id

In [ ]:
for i in platforms:
    i['platform'] = i.name
    i.insert(loc=0, column='id', value= i.name[0]+i['show_id'])

##### 1.2.11 Merge the previous four dataframes into a new one provided that they already have an ID column and a platform column

In [ ]:
df_platforms = pd.concat([df_amazon, df_disney, df_hulu, df_netflix])

##### 1.2.12 Check if the amount of unique movies in these platforms coincides with the amount of unique movies in the ratings data set

In [ ]:
len(df_ratings['movieId'].unique()) == len(df_platforms['id'].unique())

True

##### 1.2.13 Replace the null values in the rating column with the string "G"

In [ ]:
df_platforms['rating'] = df_platforms['rating'].fillna("G")

##### 1.2.14 Remove empty spaces at the beggining of the string and then use the pandas to_datetime function to convert the original string into a proper date object format

In [ ]:
df_platforms['date_added'] = df_platforms['date_added'].str.strip()
df_platforms['date_added'] = pd.to_datetime(df_platforms['date_added'], format='%B %d, %Y')

##### 1.2.15 Go through every cell, select only the string type cells and apply the lower function to those, leave the rest as they are.

In [ ]:
df_platforms.iloc[:] = df_platforms.iloc[:].applymap(lambda x: x.lower() if isinstance(x, str) else x)

##### 1.2.16 Do the same for the ratings dataframe

In [ ]:
df_ratings.iloc[:] = df_ratings.iloc[:].applymap(lambda x: x.lower() if isinstance(x, str) else x)

##### 1.2.17 I split the duration column into two new columns using the split method. Then I transform all the missing values in the duration_int column to 0 in order to be able to transform it into an integer value.

In [ ]:
df_platforms[['duration_int', 'duration_type']] = df_platforms['duration'].str.split(expand=True)
df_platforms['duration_int'] = df_platforms['duration_int'].fillna(0).astype(int)

##### 1.2.18 Rename columns

In [ ]:
df_average_score = df_average_score.rename(columns={'movieId':'id'})

##### 1.2.19 Merge both dataframes on a common columnd 'id'

In [ ]:
df_score = pd.merge(df_platforms, df_average_score, on='id')

##### 1.2.20 Rename some more columns for better readability

In [ ]:
df_score = df_score.rename(columns={'rating_x':'rating','rating_y':'score'})

Export transformed data to a new and clean dataset to be used in the API.

In [ ]:
df_score.to_csv("data_api.csv", index=False)